In [1]:
import os

In [7]:
%pwd

'c:\\Users\\alaas\\Downloads\\MLOps-MLFlow\\research'

In [8]:
os.chdir('../')

In [9]:
%pwd

'c:\\Users\\alaas\\Downloads\\MLOps-MLFlow'

In [16]:
from dataclasses import dataclass
from pathlib import Path
from src.mlops.constants import *
from src.mlops.utils.common import read_yaml, create_directories

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [34]:
# Configutation manager class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [42]:
import os
import urllib.request as request
import zipfile
from src.mlops import logger
from src.mlops.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename = os.path.basename(self.config.source_URL)
            target_file = os.path.join(self.config.unzip_dir, filename)
            
            logger.info(f"Downloading file: {filename} from {self.config.source_URL} to {target_file}")
            request.urlretrieve(
                url=self.config.source_URL,
                filename=target_file
            )
            logger.info(f"File: {filename} has been downloaded")
            
            self.extract_zip_file(target_file)
            
            os.remove(target_file)            
            logger.info(f"File: {filename} has been removed")

        else:
            logger.info(f"File: {self.config.local_data_file} already exists")
        
            
    def extract_zip_file(self, target_file):
        logger.info(f"Extracting {target_file}")
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(target_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extraction completed")

In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()

except Exception as e:
    raise e

[2024-02-16 21:08:36,998: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-16 21:08:37,002: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-16 21:08:37,005: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-16 21:08:37,008: INFO: common: created directory at: artifacts]
[2024-02-16 21:08:37,010: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-16 21:08:37,013: INFO: 2573434436: Downloading file: wine+quality.zip from https://archive.ics.uci.edu/static/public/186/wine+quality.zip to artifacts/data_ingestion\wine+quality.zip]
[2024-02-16 21:08:39,421: INFO: 2573434436: File: wine+quality.zip has been downloaded]
[2024-02-16 21:08:39,423: INFO: 2573434436: Extracting artifacts/data_ingestion\wine+quality.zip]
[2024-02-16 21:08:39,450: INFO: 2573434436: Extraction completed]
[2024-02-16 21:08:39,453: INFO: 2573434436: File: wine+quality.zip has been removed]


TypeError: DataIngestion.extract_zip_file() missing 1 required positional argument: 'target_file'

In [22]:
read_yaml(Path("config/config.yaml"))

[2024-02-16 20:55:07,068: INFO: common: yaml file: config\config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://archive.ics.uci.edu/static/public/186/wine+quality.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})